In [49]:
import numpy as np
import pandas as pd 


import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

**Загрузим данные**

In [50]:
books = pd.read_csv('input_goodreads/books.csv', encoding = "ISO-8859-1")
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [51]:
books.shape

(10000, 23)

In [52]:
books.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [53]:
ratings = pd.read_csv('input_goodreads/ratings.csv', encoding = "ISO-8859-1")
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [54]:
book_tags = pd.read_csv('input_goodreads/book_tags.csv', encoding = "ISO-8859-1")
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [55]:
tags = pd.read_csv('input_goodreads/tags.csv')
tags.tail()

,tag_id,tag_name
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ
34251,34251,ｆａｖｏｕｒｉｔｅｓ


In [56]:
tags_join_DF = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
tags_join_DF.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [57]:
tags_join_DF.sort_values('goodreads_book_id')

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
258362,1,33165,338,youth
250046,1,17213,347,kindle
246919,1,27535,348,shelfari-favorites
245888,1,16799,351,juvenile
...,...,...,...,...
918363,33288638,2541,9,angsty
956326,33288638,1126,7,5-star-reads
358501,33288638,18680,11,loved
954279,33288638,29125,10,sweet-romance


In [58]:
to_read = pd.read_csv('input_goodreads/to_read.csv')
to_read.head()

,user_id,book_id
0,1,112
1,1,235
2,1,533
3,1,1198
4,1,1874


Функция **TfidfVectorizer** из scikit-learn, которая преобразует **текст в векторы признаков**, которые можно использовать в качестве входных данных для модели.

  **Косинусное сходство** для вычисления числового значения, обозначающего сходство между двумя книгами.

In [59]:
books['authors']

0                    Suzanne Collins
1       J.K. Rowling, Mary GrandPrÃ©
2                    Stephenie Meyer
3                         Harper Lee
4                F. Scott Fitzgerald
                    ...             
9995                   Ilona Andrews
9996                  Robert A. Caro
9997                 Patrick O'Brian
9998                 Peggy Orenstein
9999                     John Keegan
Name: authors, Length: 10000, dtype: object

In [60]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(books['authors'])#(books['title'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [61]:
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

Функция, которая возвращает 20 наиболее похожих книг на основе оценки косинусного сходства.

In [62]:
titles = books['title']
indices = pd.Series(books.index, index=books['title'])

# Функция, возвращающая рекомендацию книг, основанную на cos sim авторов книг
def authors_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

In [63]:
authors_recommendations('The Hobbit').head(20)

18      The Fellowship of the Ring (The Lord of the Ri...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
188     The Lord of the Rings (The Lord of the Rings, ...
963     J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
4975        Unfinished Tales of NÃºmenor and Middle-Earth
2308                               The Children of HÃºrin
610              The Silmarillion (Middle-Earth Universe)
8271                   The Complete Guide to Middle-Earth
1128     The History of the Hobbit, Part One: Mr. Baggins
465                             The Hobbit: Graphic Novel
0                 The Hunger Games (The Hunger Games, #1)
1       Harry Potter and the Sorcerer's Stone (Harry P...
2                                 Twilight (Twilight, #1)
3                                   To Kill a Mockingbird
4                                        The Great Gatsby
5                                  The Fault in Our Stars
7             

Будем рекомендовать книги, используя теги, предоставленные для книг.

In [64]:
books_with_tags = pd.merge(books, tags_join_DF, left_on='book_id', right_on='goodreads_book_id', how='inner')

In [65]:
books_with_tags

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,goodreads_book_id,tag_id,count,tag_name
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,30574,11314,to-read
1,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11305,10836,fantasy
2,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11557,50755,favorites
3,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,8717,35418,currently-reading
4,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,33114,25968,young-adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999907,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,8914,16529,4,john-keegan
999908,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,8914,32805,3,world-war
999909,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,8914,32156,3,war-ww1
999910,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,8914,20285,3,modern-european-history


In [66]:
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [67]:
cosine_sim1

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [68]:

titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

def tags_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim1[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

In [69]:
tags_recommendations('The Hobbit').head(20)

16                  Catching Fire (The Hunger Games, #2)
31                                       Of Mice and Men
107         Confessions of a Shopaholic (Shopaholic, #1)
125                            Dune (Dune Chronicles #1)
149                                         The Red Tent
206               One for the Money (Stephanie Plum, #1)
214                                     Ready Player One
231                  The Gunslinger (The Dark Tower, #1)
253               Shiver (The Wolves of Mercy Falls, #1)
313                              Inkheart (Inkworld, #1)
325                                       White Oleander
405       The New Drawing on the Right Side of the Brain
412                                 The Three Musketeers
425                              A Confederacy of Dunces
505                          The One (The Selection, #3)
513                    The Adventures of Sherlock Holmes
525                  Darkly Dreaming Dexter (Dexter, #1)
566                            

Рекомендация книг с использованием авторов и тегов одновременно.

In [70]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...


In [71]:
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='inner')

In [72]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,to-read fantasy favorites currently-reading yo...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,to-read fantasy favorites currently-reading yo...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,to-read favorites currently-reading young-adul...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,to-read favorites currently-reading young-adul...


In [73]:
books['corpus'] = (pd.Series(books[['authors', 'tag_name']]
                .fillna('')
                .values.tolist()
                ).str.join(' '))

books['corpus'].head()

0    Suzanne Collins to-read fantasy favorites curr...
1    J.K. Rowling, Mary GrandPrÃ© to-read fantasy f...
2    Stephenie Meyer to-read fantasy favorites curr...
3    Harper Lee to-read favorites currently-reading...
4    F. Scott Fitzgerald to-read favorites currentl...
Name: corpus, dtype: object

In [74]:
tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])
cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)

titles = books['title']
indices = pd.Series(books.index, index=books['title'])

def corpus_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim_corpus[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

corpus_recommendations("The Hobbit")

188     The Lord of the Rings (The Lord of the Rings, ...
154            The Two Towers (The Lord of the Rings, #2)
160     The Return of the King (The Lord of the Rings,...
18      The Fellowship of the Ring (The Lord of the Ri...
610              The Silmarillion (Middle-Earth Universe)
4975        Unfinished Tales of NÃºmenor and Middle-Earth
2308                               The Children of HÃºrin
963     J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
465                             The Hobbit: Graphic Novel
8271                   The Complete Guide to Middle-Earth
1366    The Once and Future King (The Once and Future ...
1321              The Last Unicorn (The Last Unicorn, #1)
53      The Hitchhiker's Guide to the Galaxy (Hitchhik...
367             The Subtle Knife (His Dark Materials, #2)
61            The Golden Compass (His Dark Materials, #1)
479           The Amber Spyglass (His Dark Materials, #3)
936          His Dark Materials (His Dark Materials #1-3)
331      The M

In [75]:
corpus_recommendations("Twilight (Twilight, #1)")

51                                 Eclipse (Twilight, #3)
48                                New Moon (Twilight, #2)
991                    The Twilight Saga (Twilight, #1-4)
833                         Midnight Sun (Twilight, #1.5)
731     The Short Second Life of Bree Tanner: An Eclip...
1618    The Twilight Saga Complete Collection  (Twilig...
4087    The Twilight Saga: The Official Illustrated Gu...
2020             The Twilight Collection (Twilight, #1-3)
72                                The Host (The Host, #1)
219     Twilight: The Complete Illustrated Movie Compa...
55                           Breaking Dawn (Twilight, #4)
3074    Twilight: The Graphic Novel, Vol. 1 (Twilight:...
1802    The Awakening / The Struggle (The Vampire Diar...
2393    The Fury / Dark Reunion (The Vampire Diaries, ...
418                   Blood Promise (Vampire Academy, #4)
383                       Frostbite (Vampire Academy, #2)
4820    Jessica's Guide to Dating on the Dark Side (Je...
1108    New Mo

In [76]:
corpus_recommendations("Romeo and Juliet")

352                                           Othello
769                                     Julius Caesar
124                                            Hamlet
153                                           Macbeth
247                         A Midsummer Night's Dream
838                            The Merchant of Venice
854                                     Twelfth Night
529                            Much Ado About Nothing
713                                         King Lear
772                           The Taming of the Shrew
3947                                          Henry V
3699                                      Richard III
7073    King Henry IV, Part 1 (Wars of the Roses, #2)
386                                      The Crucible
1665                            Shakespeare's Sonnets
7                              The Catcher in the Rye
714                               Death of a Salesman
6529                                 Titus Andronicus
27                          

In [77]:
ratings

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [78]:
books

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name,corpus
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,to-read fantasy favorites currently-reading yo...,Suzanne Collins to-read fantasy favorites curr...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,"J.K. Rowling, Mary GrandPrÃ© to-read fantasy f..."
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,to-read fantasy favorites currently-reading yo...,Stephenie Meyer to-read fantasy favorites curr...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,to-read favorites currently-reading young-adul...,Harper Lee to-read favorites currently-reading...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,to-read favorites currently-reading young-adul...,F. Scott Fitzgerald to-read favorites currentl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,to-read fantasy favorites currently-reading fi...,Ilona Andrews to-read fantasy favorites curren...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,to-read favorites currently-reading books-i-ow...,Robert A. Caro to-read favorites currently-rea...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...,to-read favorites currently-reading fiction bo...,Patrick O'Brian to-read favorites currently-re...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...,to-read favorites currently-reading books-i-ow...,Peggy Orenstein to-read favorites currently-re...


Напоминание про ДЗ 1


### ДЗ 2. Content-based рекомендация.  Гибкий дедлайн 11 октября. Жесткий дедлайн 18 октября (оценка - 1 балл)

1. Приведите данные датасета ratings к виду датафрейма со строками-пользователями, столбцами-книгами и рейтингами на пересечении

user_vectors = #TO DO

(проверка: размерность датасета должна быть (53424, 10000) )

2. Создайте векторы, характеризующие пользоваталей (для content-based лучше это делать по дополнительным данным о пользователях, но так как тут у нас их нет - сделаем вектора-признаки (размерностью 100) на основе оценок) 

Делать будем с помощью метода понижения размерности PCA до 100.

3. Объедините три датасета:  
    * ratings
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе books['corpus'] для каждой книги
    
В итоге у вас должен получиться датафрейм с вектором пользователя, вектором книги и таргет-рейтинг.

4. Разбейте на train/valid set

5. Обучите любую модель машинного обучения (для задачи регрессии (линейную или ансамбль деревьев)), сделайте прогноз и посчитайте метрики RMSE, MSE
6. Добавьте другие признаки по книгам из books

7. Сделайте тестовый датасет для пользователей и книг, которые находятся в датасете to-read:
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе books['corpus'] для каждой книги
8. Сделайте для них прогноз

In [79]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [80]:
user_vectors = ratings.pivot_table(values='rating', index='user_id', columns=['book_id']).fillna(0)

In [81]:
user_vectors.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
from sklearn.decomposition import PCA

pca = PCA(100)
pca_user_vectors = pca.fit_transform(user_vectors)
assert user_vectors.shape == (53424, 10000)

# Пошаговое составление датасета для модели
Для TfidfVectorizer задано ограничение max_features=1000

In [83]:
df_pca = pd.DataFrame(pca_user_vectors)
df_pca['user_id'] = df_pca.index
df_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,user_id
0,-0.305757,-0.339938,-0.146628,-0.061094,-0.096974,-0.078034,-0.017632,-0.066789,0.007075,-0.010874,...,0.016507,-0.007004,0.003689,0.008069,-0.012095,-0.012816,-0.000918,-0.000831,0.002080,0
1,-0.313055,-0.341005,-0.148499,-0.082555,-0.112170,-0.089843,-0.011407,-0.093895,-0.005204,-0.007394,...,-0.021110,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,1
2,-0.308714,-0.333810,-0.139988,-0.079566,-0.105560,-0.087863,-0.009129,-0.088585,-0.004077,-0.001981,...,-0.018045,0.031217,0.012402,-0.011337,0.001267,-0.021648,0.000668,-0.009418,-0.006202,2
3,-0.281457,-0.260892,-0.166868,-0.017229,-0.089565,-0.040614,-0.024613,-0.068587,0.023214,-0.002050,...,-0.006734,0.071915,0.035703,-0.009048,0.056279,0.096499,-0.072210,0.039033,-0.028617,3
4,-0.315102,-0.316038,-0.093526,-0.048068,-0.132656,-0.065472,-0.033025,-0.103237,0.008036,-0.042293,...,-0.112670,0.012121,-0.095049,0.026512,-0.030462,-0.062861,0.000423,-0.089503,0.000887,4


In [84]:
df_union = pd.merge(ratings, df_pca, left_on='user_id', right_on='user_id', how='inner')
df_union.head()

,book_id,user_id,rating,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,1,314,5,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.40278,...,-0.228625,-0.037811,-0.086953,-0.372267,-0.174688,-0.669594,1.264084,-0.704521,0.23939,0.316697
1,3,314,3,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.40278,...,-0.228625,-0.037811,-0.086953,-0.372267,-0.174688,-0.669594,1.264084,-0.704521,0.23939,0.316697
2,5,314,4,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.40278,...,-0.228625,-0.037811,-0.086953,-0.372267,-0.174688,-0.669594,1.264084,-0.704521,0.23939,0.316697
3,6,314,5,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.40278,...,-0.228625,-0.037811,-0.086953,-0.372267,-0.174688,-0.669594,1.264084,-0.704521,0.23939,0.316697
4,12,314,4,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.40278,...,-0.228625,-0.037811,-0.086953,-0.372267,-0.174688,-0.669594,1.264084,-0.704521,0.23939,0.316697


In [85]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english', max_features=1000)
tfidf_matrix = tf.fit_transform(books['corpus'])
tfidf_matrix

<10000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1280338 stored elements in Compressed Sparse Row format>

In [86]:
df_tfidf = pd.DataFrame(pca_user_vectors)
df_tfidf['book_id'] = df_tfidf.index
df_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,book_id
0,-0.305757,-0.339938,-0.146628,-0.061094,-0.096974,-0.078034,-0.017632,-0.066789,0.007075,-0.010874,...,0.016507,-0.007004,0.003689,0.008069,-0.012095,-0.012816,-0.000918,-0.000831,0.002080,0
1,-0.313055,-0.341005,-0.148499,-0.082555,-0.112170,-0.089843,-0.011407,-0.093895,-0.005204,-0.007394,...,-0.021110,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,1
2,-0.308714,-0.333810,-0.139988,-0.079566,-0.105560,-0.087863,-0.009129,-0.088585,-0.004077,-0.001981,...,-0.018045,0.031217,0.012402,-0.011337,0.001267,-0.021648,0.000668,-0.009418,-0.006202,2
3,-0.281457,-0.260892,-0.166868,-0.017229,-0.089565,-0.040614,-0.024613,-0.068587,0.023214,-0.002050,...,-0.006734,0.071915,0.035703,-0.009048,0.056279,0.096499,-0.072210,0.039033,-0.028617,3
4,-0.315102,-0.316038,-0.093526,-0.048068,-0.132656,-0.065472,-0.033025,-0.103237,0.008036,-0.042293,...,-0.112670,0.012121,-0.095049,0.026512,-0.030462,-0.062861,0.000423,-0.089503,0.000887,4


# Окончательный датасет df_union

In [87]:
df_union = pd.merge(df_union, df_tfidf, left_on = "book_id", right_on = "book_id", how = 'inner')
df_union.columns

Index(['book_id', 'user_id', 'rating', '0_x', '1_x', '2_x', '3_x', '4_x',
       '5_x', '6_x',
       ...
       '90_y', '91_y', '92_y', '93_y', '94_y', '95_y', '96_y', '97_y', '98_y',
       '99_y'],
      dtype='object', length=203)

In [88]:
df_union.head()

,book_id,user_id,rating,0_x,1_x,2_x,3_x,4_x,5_x,6_x,...,90_y,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y
0,1,314,5,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.402780,...,-0.027789,-0.02111,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209
1,1,439,3,-0.285948,-0.315367,-0.145922,-0.028894,-0.060596,-0.067414,-0.061786,...,-0.027789,-0.02111,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209
2,1,588,5,3.203766,5.466464,-2.476628,-1.899373,0.859055,1.172534,-1.122889,...,-0.027789,-0.02111,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209
3,1,1169,4,-0.314882,-0.343601,-0.141867,-0.083451,-0.086767,-0.105991,-0.043367,...,-0.027789,-0.02111,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209
4,1,1185,4,0.364379,0.238944,0.127504,2.016102,1.169834,1.227979,-2.756323,...,-0.027789,-0.02111,0.070668,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209


# Train/test + модель

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [90]:
train, test = train_test_split(df_union, test_size=0.2)
model = LinearRegression().fit(train.drop('rating', axis = 1), train['rating'])
pred = model.predict(test.drop('rating', axis = 1))
mse = mean_squared_error(test['rating'], pred, squared=False)
rmse = mse**(1/2)
print("MSE = ", mse, "\nRMSE = ", rmse)

MSE =  0.9803792919628853 
RMSE =  0.990141046499379


# Добавление новых признаков

In [91]:
books.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url', 'tag_name', 'corpus'],
      dtype='object')

In [92]:
temp = books[['book_id', 'average_rating', 'work_text_reviews_count', 'work_ratings_count']] 
df_union_ultra = pd.merge(df_union, temp, 
                    on = "book_id",  
                    how = 'inner')
df_union_ultra.head()

,book_id,user_id,rating,0_x,1_x,2_x,3_x,4_x,5_x,6_x,...,93_y,94_y,95_y,96_y,97_y,98_y,99_y,average_rating,work_text_reviews_count,work_ratings_count
0,1,314,5,-0.267977,0.031997,-0.337376,-0.095021,-0.137297,0.048575,-0.402780,...,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,4.54,27520,1785676
1,1,439,3,-0.285948,-0.315367,-0.145922,-0.028894,-0.060596,-0.067414,-0.061786,...,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,4.54,27520,1785676
2,1,588,5,3.203766,5.466464,-2.476628,-1.899373,0.859055,1.172534,-1.122889,...,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,4.54,27520,1785676
3,1,1169,4,-0.314882,-0.343601,-0.141867,-0.083451,-0.086767,-0.105991,-0.043367,...,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,4.54,27520,1785676
4,1,1185,4,0.364379,0.238944,0.127504,2.016102,1.169834,1.227979,-2.756323,...,0.003345,-0.014993,0.011613,-0.043622,-0.028398,0.013578,-0.007209,4.54,27520,1785676


In [93]:
train, test = train_test_split(df_union_ultra, test_size=0.2)
model_ultra = LinearRegression().fit(train.drop('rating', axis = 1), train['rating'])
pred = model_ultra.predict(test.drop('rating', axis = 1))
mse = mean_squared_error(test['rating'], pred, squared=False)
rmse = mse**(1/2)
print("MSE = ", mse, "\nRMSE = ", rmse)

MSE =  0.9820018573710522 
RMSE =  0.9909600685048072


# Датасет to-read + предсказание на нём

In [94]:
books_to_read = pd.read_csv('input_goodreads/to_read.csv')
column_names = ["book_id", "user_id"]
books_to_read = books_to_read.reindex(columns=column_names)
books_to_read

,book_id,user_id
0,112,1
1,235,1
2,533,1
3,1198,1
4,1874,1
...,...,...
912700,4716,53424
912701,4844,53424
912702,5907,53424
912703,7569,53424


In [95]:
df_to_pred_one = pd.merge(books_to_read, df_pca, on = "user_id", how = "inner")
df_to_pred = pd.merge(df_to_pred_one, df_tfidf, on = "book_id", how = "inner")
df_to_pred.head()

,book_id,user_id,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,90_y,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y
0,112,1,-0.313055,-0.341005,-0.148499,-0.082555,-0.112170,-0.089843,-0.011407,-0.093895,...,0.179078,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.53753,0.95165,0.373169
1,112,9,-0.233918,-0.299478,-0.231114,0.037071,-0.027454,0.049149,-0.164225,0.147059,...,0.179078,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.53753,0.95165,0.373169
2,112,72,-0.304734,-0.332448,-0.186075,-0.081712,-0.091036,-0.070316,-0.002973,-0.054614,...,0.179078,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.53753,0.95165,0.373169
3,112,101,-0.311017,-0.339028,-0.139327,-0.076538,-0.099690,-0.092858,-0.010175,-0.093677,...,0.179078,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.53753,0.95165,0.373169
4,112,164,-0.310181,-0.322640,-0.135665,-0.051700,-0.125157,-0.077084,-0.028417,-0.092859,...,0.179078,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.53753,0.95165,0.373169


In [96]:
pred = model.predict(df_to_pred)

In [97]:
pred

array([3.7082917 , 3.71582312, 3.71791132, ..., 3.96975561, 3.9905505 ,
       4.01986054])

In [98]:
df_to_pred['predicted_rating'] = pd.DataFrame(pred)
df_to_pred

,book_id,user_id,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y,predicted_rating
0,112,1,-0.313055,-0.341005,-0.148499,-0.082555,-0.112170,-0.089843,-0.011407,-0.093895,...,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.537530,0.951650,0.373169,3.708292
1,112,9,-0.233918,-0.299478,-0.231114,0.037071,-0.027454,0.049149,-0.164225,0.147059,...,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.537530,0.951650,0.373169,3.715823
2,112,72,-0.304734,-0.332448,-0.186075,-0.081712,-0.091036,-0.070316,-0.002973,-0.054614,...,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.537530,0.951650,0.373169,3.717911
3,112,101,-0.311017,-0.339028,-0.139327,-0.076538,-0.099690,-0.092858,-0.010175,-0.093677,...,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.537530,0.951650,0.373169,3.710846
4,112,164,-0.310181,-0.322640,-0.135665,-0.051700,-0.125157,-0.077084,-0.028417,-0.092859,...,-0.126232,0.216999,0.055578,0.089974,-0.133413,0.343617,0.537530,0.951650,0.373169,3.714913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912680,9803,46091,-0.317642,-0.336158,-0.103713,-0.077645,-0.086874,-0.104136,-0.061025,-0.079872,...,0.463601,3.074028,-4.146307,0.067876,0.356197,1.937498,-0.548048,-2.677000,-2.992721,3.852975
912681,8490,46833,-0.371482,-0.354646,1.091378,-0.742422,-0.430958,1.603287,0.379299,-0.110412,...,0.347195,0.235384,0.050346,0.572997,-0.518841,-0.265756,-0.085167,0.675503,-0.268187,3.986674
912682,8490,48273,-0.296585,-0.339693,-0.115365,-0.081587,-0.088321,-0.100849,-0.068201,-0.061252,...,0.347195,0.235384,0.050346,0.572997,-0.518841,-0.265756,-0.085167,0.675503,-0.268187,3.969756
912683,8490,51513,-0.305127,-0.323058,-0.150319,-0.080297,-0.094222,-0.092763,-0.068346,-0.036059,...,0.347195,0.235384,0.050346,0.572997,-0.518841,-0.265756,-0.085167,0.675503,-0.268187,3.990551
